In [3]:

import cv2
import numpy as np

In [2]:

!pip3 install opencv-python
!pip3 install MIDIUtil midi2audio

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 37.3 MB 12.4 MB/s eta 0:00:01
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 1.0 MB 4.5 MB/s eta 0:00:01
  Created wheel for MIDIUtil: filename=MIDIUtil-1.2.1-py3-none-any.whl size=54567 sha256=df4971d93b8f7459d09ea871e784eb9ae9c0f1829fcce4d5eda98a547d67a79d
  Stored in directory: /Users/lucasschneider/Library/Caches/pip/wheels/52/c2/30/ce4f23db030a863977bc328651b914261581717167aabfcd47
Successfully built MIDIUtil
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


In [6]:
from midiutil import MIDIFile
from midi2audio import FluidSynth

In [11]:
import os
os.environ["PATH"] += os.pathsep + "/opt/homebrew/bin"


In [15]:
from midiutil import MIDIFile
from midi2audio import FluidSynth

# Step 1: Create MIDI
midi = MIDIFile(1)
track = 0
time = 0
midi.addTempo(track, time, 120)

channel = 0
volume = 100
duration = 1  # seconds

# Melody notes: C4, D4, E4, G4
notes = [60, 62, 64, 67]

for i, note in enumerate(notes):
    midi.addNote(track, channel, note, time + i * duration, duration, volume)

# Save MIDI file
midi_file = "melody.mid"
with open(midi_file, "wb") as output_file:
    midi.writeFile(output_file)

# Step 2: Convert MIDI to audio
soundfont = "/Users/lucasschneider/Desktop/Privat/Transcription_Project/Transcription/FluidR3_GM.sf2"
fs = FluidSynth(soundfont)
fs.midi_to_audio(midi_file, "melody.wav")

print("Done: melody.wav created.")


FluidSynth runtime version 2.4.4
Copyright (C) 2000-2025 Peter Hanappe and others.
Distributed under the LGPL license.
SoundFont(R) is a registered trademark of Creative Technology Ltd.

Rendering audio to file 'melody.wav'..
Done: melody.wav created.


fluidsynth: panic: An error occurred while reading from stdin.
fluidsynth: panic: An error occurred while reading from stdin.
